In [1]:
import pickle
import  torch
from wordcloud import WordCloud
from sklearn.naive_bayes import MultinomialNB
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [2]:
# change the data path to yours to have access to the files in your google drive
DATA_PATH = "/content/drive/MyDrive/CS247"

Reading Data from drive

In [3]:
with open(DATA_PATH + '/train_data.pkl', 'rb') as fid:
     train_data= pickle.load(fid)

with open(DATA_PATH + '/valid_data.pkl', 'rb') as fid:
     valid_data= pickle.load(fid)

with open(DATA_PATH + '/test_data.pkl', 'rb') as fid:
     test_data= pickle.load(fid)

with open(DATA_PATH + '/data.pkl', 'rb') as fid:
     data= pickle.load(fid)

In [4]:
train_data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,Target,comment_text_wp,comment_tokenize,comment
112366,591e7ceac4e56588,Would agree. While it's clear that GQ is ripe ...,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0]",would agree clear gq ripe trek references refe...,"[would, agree, clear, gq, ripe, trek, referenc...",would agree while it is clear that gq is ripe ...
108254,42aab9d0fdd419ed,WrestleMania \n\nYou son of a bitch!!! a Moron!,1,0,1,0,1,0,"[1, 0, 1, 0, 1, 0]",wrestlemania son bitch moron,"[wrestlemania, son, bitch, moron]",wrestlemania you son of a bitch a moron
57809,9abcc8c3f9280395,Larry Doby \n\nI've left some comments on that...,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0]",larry doby left comments section talk page fee...,"[larry, doby, left, comments, section, talk, p...",larry doby i have left some comments on that s...
61339,a43208fc2de19df8,"""\n\nFor kicks, the primary author of """"Cartma...",0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0]",kicks primary author cartman gets anal probe f...,"[kicks, primary, author, cartman, gets, anal, ...",for kicks the primary author of cartman gets a...
158189,ea2714389f753568,""" edit area. Do not include the """"tlx|"""" code....",0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0]",edit area include tlx code tlx unblock reason ...,"[edit, area, include, tlx, code, tlx, unblock,...",edit area do not include the tlx code tlx unbl...


In [5]:
to_join = lambda x : " ".join(x)
data['join_tokens'] = data.comment_tokenize.apply(to_join)
train_data['join_tokens'] = train_data.comment_tokenize.apply(to_join)
valid_data['join_tokens'] = valid_data.comment_tokenize.apply(to_join)
X_train = train_data['join_tokens']
X_valid = valid_data['join_tokens']
N = len(X_train)
Y_train = np.zeros((N,6))
Y_valid = np.zeros((len(X_valid),6))
for i in range(N):
  Y_train[i] = train_data['Target'].iloc[i]
for i in range(len(X_valid)):
  Y_valid[i] = valid_data['Target'].iloc[i]


unigram Bag of Words + NB

In [6]:
cv = CountVectorizer()
cv.fit(data['join_tokens'])
X_train_feature = cv.transform(X_train)
X_valid_feature  = cv.transform(X_valid)

In [7]:
from sklearn.multiclass import OneVsRestClassifier
nb = MultinomialNB()
clf = OneVsRestClassifier(nb)
clf.fit(X_train_feature,Y_train)
prob = clf.predict_proba(X_valid_feature)



In [11]:
#  mean accuracy 
mean_acc = clf.score(X_valid_feature,Y_valid)

In [12]:
mean_acc

0.8988361683079678

In [11]:
#  use tensors  for embeddings and deep learning for gpu training 
# if torch.cuda.is_available():
#   device = torch.device("cuda")
# else:
#   device = torch.device("cpu")
# from scipy.sparse import coo_matrix
# #  converting sparse bag of words to sparse tensor 
# coo = coo_matrix(X_train_feature, shape=X_train_feature.shape)
# values = coo.data
# indices = np.vstack((coo.row, coo.col))

# i = torch.LongTensor(indices)
# v = torch.FloatTensor(values)
# shape = coo.shape
# X_train_feature = torch.sparse.FloatTensor(i, v, torch.Size(shape)).cuda()

In [13]:
# coo = coo_matrix(X_valid_feature, shape=X_valid_feature.shape)
# values = coo.data
# indices = np.vstack((coo.row, coo.col))

# i = torch.LongTensor(indices)
# v = torch.FloatTensor(values)
# shape = coo.shape
# X_valid_feature = torch.sparse.FloatTensor(i, v, torch.Size(shape)).cuda()

In [15]:
# Y_train = torch.Tensor(Y_train).cuda()
# Y_valid = torch.Tensor(Y_valid).cuda()